Nome: Gabriel Meirelles Carvalho Orlando
RA: 790728


Link arquivo csv: https://drive.google.com/file/d/16TJTE8IJUUlyo5fa4BLGJiVFxpeGtSYX/view?usp=sharing

In [1]:
# Função para plotar a rede neural
# Fonte: https://stackoverflow.com/questions/29888233/how-to-visualize-a-neural-network

from matplotlib import pyplot
from math import cos, sin, atan


class Neuron():
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def draw(self, neuron_radius):
        circle = pyplot.Circle((self.x, self.y), radius=neuron_radius, fill=False)
        pyplot.gca().add_patch(circle)


class Layer():
    def __init__(self, network, number_of_neurons, number_of_neurons_in_widest_layer):
        self.vertical_distance_between_layers = 6
        self.horizontal_distance_between_neurons = 2
        self.neuron_radius = 0.5
        self.number_of_neurons_in_widest_layer = number_of_neurons_in_widest_layer
        self.previous_layer = self.__get_previous_layer(network)
        self.y = self.__calculate_layer_y_position()
        self.neurons = self.__intialise_neurons(number_of_neurons)

    def __intialise_neurons(self, number_of_neurons):
        neurons = []
        x = self.__calculate_left_margin_so_layer_is_centered(number_of_neurons)
        for iteration in range(number_of_neurons):
            neuron = Neuron(x, self.y)
            neurons.append(neuron)
            x += self.horizontal_distance_between_neurons
        return neurons

    def __calculate_left_margin_so_layer_is_centered(self, number_of_neurons):
        return self.horizontal_distance_between_neurons * (self.number_of_neurons_in_widest_layer - number_of_neurons) / 2

    def __calculate_layer_y_position(self):
        if self.previous_layer:
            return self.previous_layer.y + self.vertical_distance_between_layers
        else:
            return 0

    def __get_previous_layer(self, network):
        if len(network.layers) > 0:
            return network.layers[-1]
        else:
            return None

    def __line_between_two_neurons(self, neuron1, neuron2):
        angle = atan((neuron2.x - neuron1.x) / float(neuron2.y - neuron1.y))
        x_adjustment = self.neuron_radius * sin(angle)
        y_adjustment = self.neuron_radius * cos(angle)
        line = pyplot.Line2D((neuron1.x - x_adjustment, neuron2.x + x_adjustment), (neuron1.y - y_adjustment, neuron2.y + y_adjustment))
        pyplot.gca().add_line(line)

    def draw(self, layerType=0):
        for neuron in self.neurons:
            neuron.draw( self.neuron_radius )
            if self.previous_layer:
                for previous_layer_neuron in self.previous_layer.neurons:
                    self.__line_between_two_neurons(neuron, previous_layer_neuron)
        # write Text
        x_text = self.number_of_neurons_in_widest_layer * self.horizontal_distance_between_neurons
        if layerType == 0:
            pyplot.text(x_text, self.y, 'Input Layer', fontsize = 12)
        elif layerType == -1:
            pyplot.text(x_text, self.y, 'Output Layer', fontsize = 12)
        else:
            pyplot.text(x_text, self.y, 'Hidden Layer '+str(layerType), fontsize = 12)

class NeuralNetwork():
    def __init__(self, number_of_neurons_in_widest_layer):
        self.number_of_neurons_in_widest_layer = number_of_neurons_in_widest_layer
        self.layers = []
        self.layertype = 0

    def add_layer(self, number_of_neurons ):
        layer = Layer(self, number_of_neurons, self.number_of_neurons_in_widest_layer)
        self.layers.append(layer)

    def draw(self):
        pyplot.figure()
        for i in range( len(self.layers) ):
            layer = self.layers[i]
            if i == len(self.layers)-1:
                i = -1
            layer.draw( i )
        pyplot.axis('scaled')
        pyplot.axis('off')
        pyplot.title( 'Neural Network architecture', fontsize=15 )
        pyplot.show()

class DrawNN():
    def __init__( self, neural_network ):
        self.neural_network = neural_network

    def draw( self ):
        widest_layer = max( self.neural_network )
        network = NeuralNetwork( widest_layer )
        for l in self.neural_network:
            network.add_layer(l)
        network.draw()

In [2]:
# Vamos importar as bibliotecas necessárias
import pandas as pd
import graphviz 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
from sklearn.tree import export_graphviz
# Treino da MLP
from sklearn.neural_network import MLPClassifier

In [3]:
import numpy as np

In [4]:
# Importação
pima = pd.read_csv("house-votes-84.data", header=None)

In [5]:
#Exibe o data frame
pima

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,republican,n,n,y,y,y,y,n,n,y,y,n,y,y,y,n,y
431,democrat,n,n,y,n,n,n,y,y,y,y,n,n,n,n,n,y
432,republican,n,?,n,y,y,y,n,n,n,n,y,y,y,y,n,y
433,republican,n,n,n,y,y,y,?,?,?,?,n,y,y,y,n,y


Remove algumas instâncias para ficar um valor divisível por 10 de modo inteiro

In [6]:
pima.drop(pima.index[430:436],inplace=True)

In [7]:
pima

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,democrat,n,n,y,n,n,n,y,y,n,y,y,n,n,n,y,?
426,democrat,y,n,y,n,n,n,y,y,y,y,n,n,n,n,y,y
427,republican,n,n,n,y,y,y,y,y,n,y,n,y,y,y,n,y
428,democrat,?,?,?,n,n,n,y,y,y,y,n,n,y,n,y,y


Atributos do arquivo

In [8]:
tam_atributos = pima.shape[1]

COmo os valores precisam ser númericos, então os valores foram trocados para booleanos, e caso o valor seja NULO, que pode acontecer, é trocado para -1

In [9]:
for i in range(0, tam_atributos):
  pima[i] = pima[i].replace(['y'],1)
for i in range(1, tam_atributos):
  pima[i] = pima[i].replace(['n'],0)
for i in range(1, tam_atributos):
  pima[i] = pima[i].replace(['?'],-1)

lista = ['republican','democrat']
for i in range(0,len(lista)):
  pima[0] = pima[0].replace([lista[i]],i)

Matriz atributos

In [10]:
# Vamos separar as variáveis preditoras da variável label
feature_cols = pima.columns[1:tam_atributos]
X = pima[feature_cols]

In [11]:
X

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0,1,0,1,1,1,0,0,0,1,-1,1,1,1,0,1
1,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,-1
2,-1,1,1,-1,1,1,0,0,0,0,1,0,1,1,0,0
3,0,1,1,0,-1,1,0,0,0,0,1,0,1,0,0,1
4,1,1,1,0,1,1,0,0,0,0,1,-1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,0,0,1,0,0,0,1,1,0,1,1,0,0,0,1,-1
426,1,0,1,0,0,0,1,1,1,1,0,0,0,0,1,1
427,0,0,0,1,1,1,1,1,0,1,0,1,1,1,0,1
428,-1,-1,-1,0,0,0,1,1,1,1,0,0,1,0,1,1


In [12]:
X = X.to_numpy()

Vetor de classes


In [13]:
y = pima[0]

Transforma o dataframe em um array numpy



In [ ]:
y = y.to_numpy()

In [14]:
#Função que faz o faz o kfold
def kfoldcv(indices, k = 10):
    
    size = len(indices)
    subset_size = size // k
    subsets = [indices[x:x+subset_size] for x in range(0, len(indices), subset_size)]
    kfolds = []
    for i in range(k):
        test = subsets[i]
        train = []
        for subset in subsets:
            if subset != test:
                train.append(subset)
        kfolds.append((train,test))
        
    return kfolds

In [15]:
indices = list(range(X.shape[0]))
k = 10
#indices = list(range(10))
kf = kfoldcv(indices,k)

Une os arrays

In [16]:
def Union_train(X,y,indexes,k,j):
  #X_train
  df = list(range(k-1))
  for p in range(0, k-1):
    df[p] = X[kf[j][0][p][0]:kf[j][0][p][-1]] 
  x_train = np.concatenate(df)

  df2 = list(range(k-1))
  for p in range(0,k-1):
    df2[p] = y[kf[j][0][p][0]:kf[j][0][p][-1]]

  y_train = np.concatenate(df2)


  return x_train,y_train

Lista dos valores de teste e predição para facilitar a comparação

In [17]:
y_test = list(range(k))
X_test = list(range(k))
y_pred = list(range(k))

Cria os modelos da rede, e já os treina e testa

In [18]:
mlp = list(range(k))
for i in range(0,k):
  mlp[i] =  MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=1000)
  

for j in range(0,k):

  #Conjuntos de testes e de treinamentos
  X_train, y_train = Union_train(X,y,kf,k,j)


  X_test[j] = X[kf[j][1][0]:kf[j][1][-1]] 


  y_test[j] = y[kf[j][1][0]:kf[j][1][-1]] 

            

  mlp[j] = mlp[j].fit(X_train,y_train)

  y_pred[j] = mlp[j].predict(X_test[j])

In [22]:
from random import randint

Valor aleatorio para mostra os pesos desta rede

In [23]:
num_red = randint(0,k-1)
num_red

0

In [24]:
tam_pesos = len(mlp[num_red].coefs_)
tam_pesos

4

In [25]:
l = ['entrada','primeira','segunda','terceira','saida']

Mostra os pesos

In [26]:
print(f"Pesos da rede numero {num_red}\n")
print(f"Pesos entre a camada de {l[0]} e a {l[1]} camada escondida:")
print(mlp[num_red].coefs_[0],'\n')
for j in range(0,tam_pesos-2):
  print(f"Pesos entre a {l[j+1]} camada escondida e a {l[j+2]} camada escondia: ")
  print(mlp[num_red].coefs_[j+1],"\n")

print(f"Pesos entre a {l[3]} camada escondida e a camada de {l[4]}")
print(mlp[num_red].coefs_[3])

Pesos da rede numero 0

Pesos entre a camada de entrada e a primeira camada escondida:
[[ 0.31635596  0.12016271 -0.32404318 -0.22600804 -0.17361244 -0.21467294
   0.11157089 -0.08125512]
 [ 0.16186929  0.42856795 -0.33436113  0.39242739  0.49067072 -0.10530914
   0.29583428 -0.1411904 ]
 [ 0.03579133 -0.03210604 -0.45797448 -0.48325473  0.56641898  0.49051988
   0.2555227  -0.38732364]
 [ 0.24084132 -0.65392263  0.50609252  0.55960457 -0.67048173 -0.27375848
   0.98643894  0.21313858]
 [ 0.45421331 -0.31568182 -0.61369654 -0.09189054  0.45491442 -0.13142036
  -0.23745252  0.15116003]
 [ 0.15538883  0.19573106 -0.14434406 -0.02992686  0.15983374  0.05369218
   0.1991124   0.02348542]
 [ 0.52914284 -0.3634564  -0.25912867  0.06150633  0.36015582  0.71783887
   0.31386867  0.64099162]
 [-0.15393019 -0.0530131   0.42923598 -0.55590642  0.38282968  0.18081273
   0.05244379  0.38967812]
 [-0.12826185  0.77859167 -0.17899775 -0.42019639 -0.12698111  0.27489896
  -0.02007638  0.02168679]
 [-0

Função de ativação da camada de saída

In [27]:
print(f"Função de ativação na camada de saída da rede {num_red}:")
print(mlp[num_red].out_activation_)

Função de ativação na camada de saída da rede 0:
logistic


In [21]:
#Importanto novas bibliotecas
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

Função para avaliar a matriz de confusão

In [28]:
def evaluation(vector,classe):
  classes = ['republican','democrat']
  print(f"Classe {classes[classe]}:\n")
  print(f"{vector[classe]} amostras foram previstos certos.")
  soma = vector.sum()
  errados = abs(vector[classe] - soma)
  if(errados == 0):
    print("Todas as amostrar foram previstas corretamente\n")
  else:
    print(f"{errados} amostras foram previstas incorretamente\n")
  

Imprime o dicionario gerado por classification report

In [29]:
def print_dicionario(dicionario):

  nomes = []
  for k in dicionario.keys():
    nomes.append(k)
  
  print("           precision   recall    f1-score    support\n")
  for j in nomes:
    if(j != 'accuracy'):
      print(f"{j:>11}      {dicionario[j]['precision']:.2f}       {dicionario[j]['recall']:.2f}       {dicionario[j]['f1-score']:.2f}    {dicionario[j]['support']:.0f}")
    else:
      space = ' '
      print(f"{j}{31*space}{dicionario[j]:.2f}    {dicionario['macro avg']['support']:.0f}")

  print("\n\n")

Faz a impressão do classification report e da matriz de confusão

Para facilitar a visualização novamente foram inseridos os nomes reais das classes, visando permitir uma maior interpretação dos resultados

In [ ]:
target_names = ['republican','democrat']
#Vetores para a analise
precisao = [[],[],[],[]]
recall = [[],[],[],[]]
f1_score = [[],[],[],[],[]]
tam_classe = len(target_names)
for i in range(0,k):
  print(f"Para o Fold {i}:")

  c = confusion_matrix(y_test[i], y_pred[i])
  classification = classification_report(y_test[i], y_pred[i], target_names=target_names,output_dict=True)

 
  print_dicionario(classification)

  for l in range(0,tam_classe):
    evaluation(c[l],l)

  
 


  #Analise

  for p in range(0,tam_classe):
    precisao[p].append(classification[target_names[p]]['precision'])
    recall[p].append(classification[target_names[p]]['recall'])
    f1_score[p].append(classification[target_names[p]]['f1-score'])

  f1_score[tam_classe].append(classification['accuracy'])

  resto = ['macro avg','weighted avg']
  for p in range(tam_classe,len(f1_score)-1):
      precisao[p].append(classification[resto[p-2]]['precision'])
      recall[p].append(classification[resto[p-2]]['recall'])
      f1_score[p+1].append(classification[resto[p-2]]['f1-score'])

In [ ]:
arr = np.array(precisao)
arr2 = np.array(recall)
arr3 = np.array(f1_score)

In [ ]:
rotulos = ['republican','democrat','accuracy','macro avg','weighted avg']

Calcula as médias e os desvios padrões

In [ ]:
arrange = 0
tam = len(rotulos)
for x in range(0,tam):
  if(rotulos[x] != 'accuracy'):
    print(f"Media da precisão da {rotulos[x]}: {arr[x+arrange].mean()}")
    print(f"Desvio padrão da precisão da {rotulos[x]}: {arr[x+arrange].std()}\n")

    print(f"Media do recall da {rotulos[x]}: {arr2[x+arrange].mean()}")
    print(f"Desvio padrão do recall da {rotulos[x]}: {arr2[x+arrange].std()}\n")
  else:
    arrange = -1

  print(f"Media do f1-score da {rotulos[x]}: {arr3[x].mean()}")
  print(f"Desvio padrão do f1-score da {rotulos[x]}: {arr3[x].std()}\n")

  print("--------------------------------------------------------------------------")

Media da precisão da republican: 0.9311954406962147
Desvio padrão da precisão da republican: 0.056521255842999406

Media do recall da republican: 0.9428390688259111
Desvio padrão do recall da republican: 0.06711482767342643

Media do f1-score da republican: 0.9340899082810846
Desvio padrão do f1-score da republican: 0.03605023492265942

--------------------------------------------------------------------------
Media da precisão da democrat: 0.9669879475363345
Desvio padrão da precisão da democrat: 0.037713733048098164

Media do recall da democrat: 0.9529647435897436
Desvio padrão do recall da democrat: 0.048125042692331435

Media do f1-score da democrat: 0.9585582155525835
Desvio padrão do f1-score da democrat: 0.025823598147590533

--------------------------------------------------------------------------
Media do f1-score da accuracy: 0.95
Desvio padrão do f1-score da accuracy: 0.029063227656508806

--------------------------------------------------------------------------
Media da p